In [1]:
import cv2 as cv
import numpy as np

if not cv.useOptimized():
    cv.setUseOptimized(True)

cv.useOptimized()

True

In [2]:
from utils import display_image, display_images

# Weekly activity
###### 1)Rotate image by 45 degrees without cropping the sides of the image. (Hint: There are 2 strategies to tackle these problems). Use "lena.jfif" as the input image.
- Use external libraries imutils.
- Modify the transformation matrix.

In [35]:
# first method
import imutils
img = cv.imread('images/lena.jfif')
rotated_img = imutils.rotate_bound(img,45)
display_image("rotate 45",rotated_img)

In [34]:
# second method
img = cv.imread('images/lena.jfif')
center = (w // 2, h // 2)
M = cv.getRotationMatrix2D(center, -45, 1)
cos = np.abs(M[0, 0])
sin = np.abs(M[0, 1])
new_w = int((h * sin) + (w * cos))
new_h = int((h * cos) + (w * sin))
M[0, 2] += (new_w / 2) - center[0]
M[1, 2] += (new_h / 2) - center[1]
rotated_img = cv.warpAffine(img, M, (new_w, new_h))
display_image("Rotated Image (45 degrees)", rotated_img)

##### 2)Use the images with titles: "flower.jfif" and "native-bee.png". I want to put flower above an image. If I add two images, it will change color. If I blend it, I get a transparent effect. But I want it to be opaque. If it was a rectangular region, we could use the ROI as we did in the previous section. But flower is not a rectangular region. This is where bitwise operations, like AND, OR, NOT and XOR really come in handy. The associated functions are cv.bitwise_and(), cv.bitwise_or() and cv.bitwise_not(). You need to use cv.threshold function to segment the flower. Please refer to online documentation for more info. The result should resemble the following:

In [49]:
flower = cv.imread('images/flower.jfif')
bee = cv.imread('images/native-bee.png')

flower_gray = cv.cvtColor(flower, cv.COLOR_BGR2GRAY)

# Create a binary mask of the flower
_, mask = cv.threshold(flower_gray, 75, 255, cv.THRESH_BINARY)

# Invert the mask
mask_inv = cv.bitwise_not(mask)

# Define the top-left corner of where the flower will be placed on the bee image
top_left_x = 0
top_left_y = 0

flower_height, flower_width = flower.shape[:2]

# Get ROI on the bee image where the flower will be placed
roi = bee[top_left_y:top_left_y + flower_height, top_left_x:top_left_x + flower_width].copy()

# Extract the flower region from the flower image using the mask
flower_fg = cv.bitwise_and(flower, flower, mask=mask)

# Extract the background region from the bee image using the inverted mask
bee_bg = cv.bitwise_and(roi, roi, mask=mask_inv)

# Combine the flower foreground and the bee background
result_roi = cv.add(bee_bg, flower_fg)

# Place the combined ROI back into the original bee image
bee[top_left_y:top_left_y + flower_height, top_left_x:top_left_x + flower_width] = result_roi
display_image("Combined Image", result_roi)

##### 3)Write a function that randomly crop the central region of an image. The method signature should be as shown in the following:
- random_center_crop(image, min_crop_ratio, max_crop_ratio)

In [56]:
def random_center_crop(image, min_crop_ratio, max_crop_ratio):
    """
    Randomly crop the central region of an image.

    Parameters:
    - image: Input image (numpy array).
    - min_crop_ratio: Minimum ratio for cropping (0.0 to 1.0).
    - max_crop_ratio: Maximum ratio for cropping (0.0 to 1.0).

    Returns:
    - Cropped image (numpy array).
    """
    if min_crop_ratio < 0.0 or min_crop_ratio > 1.0:
        raise ValueError("min_crop_ratio must be between 0.0 and 1.0")
    elif max_crop_ratio < 0.0 or max_crop_ratio > 1.0:
        raise ValueError("max_crop_ratio must be between 0.0 and 1.0")
    elif min_crop_ratio > max_crop_ratio:
        raise ValueError("min_crop_ratio must not be greater than max_crop_ratio")

    height, width = image.shape[:2]

    min_crop_size = int(min(height, width) * min_crop_ratio)
    max_crop_size = int(min(height, width) * max_crop_ratio)

    # Randomly select crop size
    crop_size = np.random.randint(min_crop_size, max_crop_size + 1)

    # Calculate crop region
    start_x = max(0, (width - crop_size) // 2)
    start_y = max(0, (height - crop_size) // 2)
    end_x = start_x + crop_size
    end_y = start_y + crop_size

    # Perform crop
    cropped_image = image[start_y:end_y, start_x:end_x]

    return cropped_image

img = random_center_crop(cv.imread("images\lena.jfif"), 0.1, 0.5)
display_image("result",img)

##### 4)Aside from Gaussian noise, name another common type of noise. Write the code to demonstrate how the noise can be included in an image.

In [61]:
def add_salt_and_pepper_noise(image, salt_prob, pepper_prob):
    """
    Add salt and pepper noise to an image.
    Parameters:
    - Input image (numpy array).
    - salt_prob (float): Probability of adding salt noise to a pixel (0 to 1).
    - pepper_prob (float): Probability of adding pepper noise to a pixel (0 to 1).
    
    Returns:
    Image (numpy array) with added salt and pepper noise.
    """
    noisy_image = np.copy(image)
    
    # Salt noise
    salt_mask = np.random.random(image.shape[:2]) < salt_prob
    noisy_image[salt_mask] = 255
    # Pepper noise
    pepper_mask = np.random.random(image.shape[:2]) < pepper_prob
    noisy_image[pepper_mask] = 0
    return noisy_image

In [62]:
img = cv.imread("images/dog.jfif")
noisy_image = add_salt_and_pepper_noise(img,0.05,0.1)
display_images([img,noisy_image],("Original","Noisy_Image"))